# Image Classification Integration with AI Squared

In this notebook, we will show how AI Squared can be utilized in existing data science workflows to integrate AI into browser-based workflows easily and seamlessly.  We will be utilizing the [UTKFace Dataset](https://susanqq.github.io/UTKFace/) to build an age classification model which classifies headshots of individuals by their age, by decade.

We then utilize the [AI Squared Python Package](https://pypi.org/project/aisquared) to build a `.air` file, which is the format of file to be integrated into the [AI Squared Inference Engine/Chrome Extension](https://chrome.google.com/webstore/detail/ai2-extension/fgoldjnnlomijbkeeendibobbkdabjmh).

In [0]:
# Import packages
import tensorflow as tf
import numpy as np
import zipfile
import mlflow
import os

# Import the aisquared package
import aisquared

# Data Preparation

In the following cell, we set some important parameters for data preparation and augmentation to assist with training.  Additionally, we create a function to load the data into a generator.

In [0]:
# Data file is already stored in the DBFS
data_file = '/dbfs/FileStore/tables/utkface_split.zip'


# Some parameters
model_save_location = 'model.h5'
model_name = 'AgePredictor'

image_size = (256, 256)
batch_size = 256
rotation = 1/20
starting_filters = 32
neurons = 256
epochs = 100


# Unzip the data if needed
if not os.path.exists('utkface-split'):
    with zipfile.ZipFile(data_file) as f:
        f.extractall()

# Build a data generator function
def data_generator(directory, batch_size):
    files = os.listdir(directory)
    np.random.shuffle(files)
    
    cutoffs = list(range(10, 100, 10))
    
    i = 0
    while True:
        batch, labels = [], []
        for _ in range(batch_size):
            if i >= len(files):
                np.random.shuffle(files)
                i = 0
            img = tf.keras.preprocessing.image.load_img(
            os.path.join(directory, files[i]),
                target_size = image_size
            )
            img = np.array(img)
            age = int(files[i].split('_')[0])
            label = sum([age > cutoff for cutoff in cutoffs])
            batch.append(img)
            labels.append(label)
            i += 1
        yield np.asarray(batch), np.asarray(labels)

# Model Creation

In this cell, we create the model that will be trained.  This model consists of two individual models in sequence.  The first model performs data preprocessing and augmentation, including a rescaling of pixel values to `[0, 1]`, random horizontal flips of input images, and random rotation of images.  The second model contains the decision logic, containing multiple convolutional blocks which are then fed into a fully connected architecture for classification.

In [0]:
# Create the preprocessing model
preproc_model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.RandomFlip('horizontal'),
        tf.keras.layers.RandomRotation(rotation)
    ]
)

# Create the decision model
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(starting_filters, 3, padding = 'same', activation = 'relu', input_shape = (256, 256, 3)),
        tf.keras.layers.Conv2D(starting_filters, 3, padding = 'same', activation = 'relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(starting_filters * 2, 3, padding = 'same', activation = 'relu'),
        tf.keras.layers.Conv2D(starting_filters * 2, 3, padding = 'same', activation = 'relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(starting_filters * 4, 3, padding = 'same', activation = 'relu'),
        tf.keras.layers.Conv2D(starting_filters * 4, 3, padding = 'same', activation = 'relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(starting_filters * 8, 3, padding = 'same', activation = 'relu'),
        tf.keras.layers.Conv2D(starting_filters * 8, 3, padding = 'same', activation = 'relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(neurons, activation = 'relu'),
        tf.keras.layers.Dense(neurons, activation = 'relu'),
        tf.keras.layers.Dense(neurons, activation = 'relu'),
        tf.keras.layers.Dense(10, activation = 'softmax')
    ]
)


# Create the model that is to be trained as the sequence of the preprocessing and decision models
to_train = tf.keras.models.Sequential(
    [
        preproc_model,
        model
    ]
)

# Compile the model to be trained
to_train.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Fit the model

In this cell, we fit the model using early stopping

In [0]:
to_train.fit(
    data_generator('utkface-split/train/', batch_size),
    epochs = epochs,
    callbacks = tf.keras.callbacks.EarlyStopping('val_loss', patience = 3, min_delta = 0.005),
    steps_per_epoch = len(os.listdir('utkface-split/train'))//batch_size,
    validation_data = data_generator('utkface-split/validation', batch_size),
    validation_steps = len(os.listdir('utkface-split/validation'))//batch_size
)

Epoch 1/100
 1/74 [..............................] - ETA: 17:01 - loss: 2.3030 - accuracy: 0.0938 2/74 [..............................] - ETA: 4:42 - loss: 2.2322 - accuracy: 0.2168  3/74 [>.............................] - ETA: 4:38 - loss: 3.8205 - accuracy: 0.2630 4/74 [>.............................] - ETA: 4:34 - loss: 3.3736 - accuracy: 0.2432 5/74 [=>............................] - ETA: 4:30 - loss: 3.1412 - accuracy: 0.2313 6/74 [=>............................] - ETA: 4:26 - loss: 2.9936 - accuracy: 0.2253 7/74 [=>............................] - ETA: 4:21 - loss: 2.8895 - accuracy: 0.2221 8/74 [==>...........................] - ETA: 4:17 - loss: 2.8097 - accuracy: 0.2183 9/74 [==>...........................] - ETA: 4:13 - loss: 2.7440 - accuracy: 0.217910/74 [===>..........................] - ETA: 4:09 - loss: 2.6809 - accuracy: 0.217211/74 [===>..........................] - ETA: 4:05 - loss: 2.6217 - accuracy: 0.215212/74 [===>..........................] - ETA: 4:01 - loss: 2.5869 - accuracy: 0.221413/74 [====>.........................] - ETA: 3:58 - loss: 2.5452 - accuracy: 0.228114/74 [====>.........................] - ETA: 3:53 - loss: 2.4994 - accuracy: 0.238315/74 [=====>........................] - ETA: 3:49 - loss: 2.4651 - accuracy: 0.244816/74 [=====>........................] - ETA: 3:45 - loss: 2.4345 - accuracy: 0.250017/74 [=====>........................] - ETA: 3:41 - loss: 2.4057 - accuracy: 0.255718/74 [======>.......................] - ETA: 3:38 - loss: 2.3825 - accuracy: 0.258719/74 [======>.......................] - ETA: 3:34 - loss: 2.3614 - accuracy: 0.262120/74 [=======>......................] - ETA: 3:30 - loss: 2.3401 - accuracy: 0.265821/74 [=======>......................] - ETA: 3:26 - loss: 2.3224 - accuracy: 0.268622/74 [=======>......................] - ETA: 3:22 - loss: 2.3021 - accuracy: 0.271823/74 [========>.....................] - ETA: 3:18 - loss: 2.2850 - accuracy: 0.275124/74 [========>.....................] - ETA: 3:14 - loss: 2.2672 - accuracy: 0.2773

# Evaluate the model

In this cell, we evaluate the model on validation data

In [0]:
loss, accuracy = to_train.evaluate(data_generator('utkface-split/validation/', batch_size), steps = len(os.listdir('utkface-split/validation'))//batch_size)
model.save(model_save_location)
model.summary()

1/18 [>.............................] - ETA: 19s - loss: 1.0093 - accuracy: 0.6016 2/18 [==>...........................] - ETA: 17s - loss: 1.0157 - accuracy: 0.5684 3/18 [====>.........................] - ETA: 16s - loss: 1.0468 - accuracy: 0.5573 4/18 [=====>........................] - ETA: 15s - loss: 1.0316 - accuracy: 0.5654 5/18 [=======>......................] - ETA: 14s - loss: 1.0301 - accuracy: 0.5648 6/18 [=========>....................] - ETA: 12s - loss: 1.0179 - accuracy: 0.5775 7/18 [==========>...................] - ETA: 11s - loss: 1.0215 - accuracy: 0.5798 8/18 [============>.................] - ETA: 10s - loss: 1.0132 - accuracy: 0.5845 9/18 [==============>...............] - ETA: 9s - loss: 1.0074 - accuracy: 0.5846 10/18 [===============>..............] - ETA: 8s - loss: 0.9988 - accuracy: 0.590211/18 [=================>............] - ETA: 7s - loss: 0.9939 - accuracy: 0.586612/18 [===================>..........] - ETA: 6s - loss: 1.0080 - accuracy: 0.582413/18 [====================>.........] - ETA: 5s - loss: 1.0083 - accuracy: 0.580814/18 [======================>.......] - ETA: 4s - loss: 1.0072 - accuracy: 0.581215/18 [========================>.....] - ETA: 3s - loss: 1.0095 - accuracy: 0.582616/18 [=========================>....] - ETA: 2s - loss: 1.0089 - accuracy: 0.582017/18 [===========================>..] - ETA: 1s - loss: 1.0108 - accuracy: 0.584318/18 [==============================] - ETA: 0s - loss: 1.0024 - accuracy: 0.587218/18 [==============================] - 19s 1s/step - loss: 1.0024 - accuracy: 0.5872
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "sequential_1"
_________________________________________________________________
 Layer (type) Output Shape Param # 
=================================================================
 conv2d (Conv2D) (None, 256, 256, 32) 896 
 
 conv2d_1 (Conv2D) (None, 256, 256, 32) 9248 
 
 max_pooling2d (MaxPooling2D (None, 128, 128, 32) 0 
 ) 
 
 conv2d_2 (Conv2D) (None, 128, 128, 64) 18496 
 
 conv2d_3 (Conv2D) (None, 128, 128, 64) 36928 
 
 max_pooling2d_1 (MaxPooling (None, 64, 64, 64) 0 
 2D) 
 
 conv2d_4 (Conv2D) (None, 64, 64, 128) 73856 
 
 conv2d_5 (Conv2D) (None, 64, 64, 128) 147584 
 
 max_pooling2d_2 (MaxPooling (None, 32, 32, 128) 0 
 2D) 
 
 conv2d_6 (Conv2D) (None, 32, 32, 256) 295168 
 
 conv2d_7 (Conv2D) (None, 32, 32, 256) 590080 
 
 ma

# Create the `.air` file

Now that the model itself has been created and saved, we can create the `.air` file that can be used to integrate the model into the browser workflow.  The configuration of the `.air` file includes steps to determine data harvesting, data preprocessing, the analytic or model that is to be run on the data, result postprocessing and rendering, as well as how users can input feedback into the model.

Once the configuration object is created, the `.compile()` method aggregates and stores the configuration as well as a converted version of the model itself into the `.air` package.  It is important to note in this case that the model itself is stored in this file, meaning that when someone runs this file in the extension, the entire analytic workflow happens **locally** - no need to deploy the model to a remote resource (although we support that too).

In [0]:
# Create the label map that configures class names in a natural language context
label_map = [
        '0-10',
        '11-20',
        '21-30',
        '31-40',
        '41-50',
        '51-60',
        '61-70',
        '71-80',
        '81-90',
        '>90',
    ]

# Configure images to be harvested
harvester = aisquared.config.harvesting.ImageHarvester()

# Set up preprocessing steps for images to be brought into the model
  # Steps include reszing image to model input shape and normalizing by dividing by 255
preprocesser = aisquared.config.preprocessing.image.ImagePreprocessor(
    [
        aisquared.config.preprocessing.image.Resize([256, 256]),
        aisquared.config.preprocessing.image.DivideValue(255)
    ]
)

# Set up the analytic to be run, in this case a local model
analytic = aisquared.config.analytic.LocalModel(model_save_location, 'cv')

# Set up the postprocesser, a multiclass classification output
postprocesser = aisquared.config.postprocessing.MulticlassClassification(
    label_map
)

# Set up rendering, which in this case is rendering predictions on images
renderer = aisquared.config.rendering.ImageRendering(thickness = '2', font_size = '10')

# Set up a multiclass classification feedback loop
feedback = aisquared.config.feedback.MulticlassFeedback(label_map)

# Create the configuration object
config = aisquared.config.ModelConfiguration(
    model_name,
    harvester,
    preprocesser,
    analytic,
    postprocesser,
    renderer,
    feedback
)

# Compile the object into the .air file
config.compile()

WARNING:tensorflow:No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.

# Logging with MLFlow

Finally, we log important parameters, metrics, and the model and `.air` files themselves utilizing [MLFlow](https://mlflow.org)

In [0]:
with mlflow.start_run():
    mlflow.log_param('rotation', rotation)
    mlflow.log_param('image size', image_size)
    mlflow.log_param('starting filters', starting_filters)
    mlflow.log_param('neurons', neurons)
    mlflow.log_param('batch size', batch_size)
    mlflow.log_param('epochs', epochs)
    mlflow.log_metric('validation loss', loss)
    mlflow.log_metric('validation accuracy', accuracy)
    mlflow.log_artifact('model.h5')
    mlflow.log_artifact(f'{model_name}.air')